## Quadrature

This is a notebook that illustrates differences in the quadrature methods which are central to the evaluation of

$$
\int_0^{\pi} f(\theta) \sin \theta \; \mathrm{d}\theta = \int_{-1}^{1} f(\cos \theta) \; \mathrm{d}\cos \theta.
$$

Quadrature is used to compute the projection onto the associated Legendre functions in `paddle-harmonics`.

In order to illustrate how interpolation and quadrature affect the error in the computation of the SHT, this notebook contains example for both errors.

In [ ]:
import numpy as np
import paddle
import scipy
import matplotlib.pyplot as plt

In [ ]:
n_theta = 80

In [ ]:
import paddle_harmonics
from paddle_harmonics.quadrature import *

### Test interpolation

we first assess the interpolation onto the quadrature nodes:

In [ ]:
# interpolation - careful - this breaks if points align (for non-periodic signals)
def interpolate(t, tq, f):
    j = np.searchsorted(t, tq) - 1
    d = paddle.to_tensor( (tq - t[j]) / np.diff(t)[j] )
    j = paddle.to_tensor(j)
    interp = paddle.lerp(paddle.to_tensor(f[j]), paddle.to_tensor(f[j+1]), d)
    # interp = f[j] + (f[j+1] - f[j]) * (tq - t[j]) / np.diff(t)[j]
    # print(d)
    # print(f[j+1] - f[j])
    # print(j)
    return interp

In [ ]:
cost_lg, wlg = legendre_gauss_weights(n_theta, -1, 1)
# cost_lg, wlg = lobatto_weights(n_theta, -1, 1)
tq = np.flip(np.arccos(cost_lg))
teq = np.linspace(0, np.pi, n_theta, dtype=np.float64)

In [ ]:
plt.plot(teq, '.')
plt.plot(tq, '.')
plt.show()

test interpolation:

In [ ]:
f = lambda t : np.cos(4*t)
# f = lambda t : 1 / (1 + 25 * (2*(t-np.pi/2)/np.pi)**2)
# f = lambda t : 1 / (1 + 25 * np.cos(t)**2)
# f = lambda t : t**5 - 3*t**2 - 2*t + 1.0

interp = interpolate(teq, tq, f(teq))

In [ ]:
interp

In [ ]:
f(teq)

In [ ]:
# plt.plot(teq, f(teq), '.-', label="reference")
plt.plot(tq, f(tq), '.-', label="reference")
# plt.plot(tq, interp, '.-', label="interpolated")
plt.legend()
plt.show()

### Test quadrature with associated Legendre polynomials

let us test different quadrature modes:

In [ ]:
def precompute_legpoly(m_max, l_max, x):
    """
    Computes the values of P^m_n(\cos \theta) at the positions specified by x (theta)
    The resulting tensor has shape (m_max, l_max, len(x))
    """

    # compute the tensor P^m_n:
    pct = np.zeros((m_max, l_max, len(x)), dtype=np.float64)

    sinx = np.sin(x)
    cosx = np.cos(x)

    a = lambda m, l: np.sqrt((4*l**2 - 1) / (l**2 - m**2))
    b = lambda m, l: -1 * np.sqrt((2*l+1)/(2*l-3)) * np.sqrt(((l-1)**2 - m**2)/(l**2 - m**2))

    # start by populating the diagonal and the second higher diagonal
    amm = np.sqrt( 1. / (4 * np.pi) )
    pct[0,0,:] = amm
    pct[0,1,:] = a(0, 1) * cosx * amm
    for m in range(1, min(m_max, l_max)):
        pct[m,m,:] = -1*np.sqrt( (2*m+1) / (2*m) ) * pct[m-1,m-1,:] * sinx
        if m + 1 < l_max:
            pct[m,m+1,:] = a(m, m+1) * cosx * pct[m,m,:]

    # fill the remaining values on the upper triangle
    for m in range(0, m_max):
        for l in range(m+2, l_max):
            pct[m,l,:] = a(m,l) * cosx * pct[m,l-1,:] + b(m,l) * pct[m,l-2,:]

    return paddle.to_tensor(pct)

Let us plot the Legendre polynomials:

In [ ]:
m = 0

pct = np.sqrt(2 * np.pi) * precompute_legpoly(n_theta, n_theta, teq)

fig, ax = plt.subplots(1, 1)
for l in range(6):
    ax.plot(np.cos(teq), pct[0, l].numpy())
fig.show()

In [ ]:
def project(t, w, f, mmax=None):
    m = 0
    if mmax == None:
        mmax = len(t)

    weights = paddle.to_tensor(w)
    pct = np.sqrt(2 * np.pi) * precompute_legpoly(mmax, mmax, t)
    weights = paddle.einsum('mlk,k->mlk', pct, weights)

    proj = paddle.einsum('...k,lk->...l', paddle.to_tensor(f), weights[m])
    rec = paddle.einsum('...l, lk->...k', proj, pct[m] )
    return rec

let us compare the accuracy of the different projection methods:

In [ ]:
t = np.linspace(0, np.pi, n_theta)
plt.plot(t, f(t), label="reference")

for quadrature in [legendre_gauss_weights, lobatto_weights, clenshaw_curtiss_weights, fejer2_weights]:
    cost, wq = quadrature(n_theta, -1, 1)
    tq = np.flip(np.arccos(cost))

    out = project(tq, wq, f(tq))

    plt.plot(tq, out.numpy(), '.-', label=quadrature.__name__)

plt.legend(loc='lower left')
plt.show()

In [ ]:
for quadrature in [legendre_gauss_weights, lobatto_weights, clenshaw_curtiss_weights]:
    cost, wq = quadrature(n_theta, -1, 1)
    tq = np.flip(np.arccos(cost))

    out = project(tq, wq, f(tq))
    # print(np.abs(out - f(tq)))

    plt.semilogy(tq, out.numpy() - f(tq), '.-', label=quadrature.__name__)

plt.legend(loc='lower left')
plt.show()

Let us now add interpolation into the mix to evaluate performance with interpolation taken into account. For this particular case, we will assume that the data is given to us on an equidistant grid.

In [ ]:
t = np.linspace(0, np.pi, n_theta)
ref = f(t)
plt.plot(t, ref, label="reference")

for quadrature in [legendre_gauss_weights, lobatto_weights, clenshaw_curtiss_weights]:
    cost, wq = quadrature(n_theta, -1, 1)
    tq = np.flip(np.arccos(cost))

    if quadrature == lobatto_weights or quadrature == legendre_gauss_weights:
        f_interp = interpolate(t, tq, ref)
        mmax = len(tq)
    else:
        f_interp = ref
        mmax = len(tq)

    out = project(tq, wq, f_interp, mmax=mmax)

    plt.plot(tq, out, '.-', label=quadrature.__name__)

plt.legend(loc='lower left')
plt.show()

again, let us plot the overall error, this time including the interpolation error:

In [ ]:
t = np.linspace(0, np.pi, n_theta)
ref = f(t)

fig, ax = plt.subplots(2, 1)

for quadrature in [legendre_gauss_weights, lobatto_weights, clenshaw_curtiss_weights]:
    cost, wq = quadrature(n_theta, -1, 1)
    tq = np.flip(np.arccos(cost))

    if quadrature == lobatto_weights or quadrature == legendre_gauss_weights:
        f_interp = interpolate(t, tq, ref)
        mmax = len(tq)
    else:
        f_interp = ref
        mmax = len(tq)

    out = project(tq, wq, f_interp, mmax=mmax)

    ax[0].semilogy(tq, out.numpy() - f(tq), '.-', label=quadrature.__name__)
    if isinstance(f_interp, paddle.Tensor):
        f_interp = f_interp.numpy()
    ax[1].semilogy(tq, f_interp - f(tq), '.-', label=quadrature.__name__)

ax[0].set_title("Projection error after interpolation")
ax[1].set_title("Interpolation error")
# ax[0].legend(loc='lower left')
# ax[1].legend(loc='lower left')
fig.tight_layout()
plt.show()

we can see that the interpolation dominates when we interpolate the solution. For this reason, it is reasonable t choose Clenshaw-Curtiss quadrature in scenarios where we expect the interpolation error to dominate.